In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import h5py
from tensorflow.keras.models import load_model
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#################### Importing wheigts #####################################
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications import mobilenet_v3
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications import NASNetLarge, Xception
#########################################################

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint, CSVLogger
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
import pandas as pd

In [2]:
import gc
gc.collect()

22

In [3]:
image_size = 224
def_labels = ['glioma_tumor','meningioma_tumor','pituitary_tumor']

In [ ]:
"ResNet152V2 Implementation : Best Model"
ResNet152V2 = ResNet152V2(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))
model = ResNet152V2.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=ResNet152V2.input, outputs = model)

In [ ]:
opt = SGD(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer = opt, metrics= ['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs')
csv_logger = CSVLogger('/kaggle/working/ResNet152V2.csv',separator=",")
checkpoint = ModelCheckpoint("ResNet152V2.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.5, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [ ]:
Train_Dir = 'C:\\Users\\Eurus\\Desktop\\B.Sc_Project\\Data\\Training'
Test_Dir = 'C:\\Users\\Eurus\\Desktop\\B.Sc_Project\\Data\\Testing'

In [ ]:
GT_features = {}
root = 'C:\\Users\\Eurus\\Desktop\\B.Sc_Project\\Data\\Training\\glioma_tumor'
for idx in os.listdir(root):
    # Read them from their path
    path = os.path.join(root,idx)
    # Load each one of them
    img = image.load_img(path, target_size=(224, 224))
    # Prep them 
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    # Extract the features
    features = EfficientNetB0.predict(img_data)
    """Faltten them all and then append them to a dictionary"""    
    features = features.flatten()
    GT_features[idx] = features
    

In [ ]:
len(GT_features['GLIOMA_TUMOR_TR_AUG__0_1334593.jpg'])

In [ ]:
GT_features_train = pd.DataFrame()
for key, value in GT_features.items():
    temp_df = pd.DataFrame(value)
    temp_df['key'] = key
    GT_features_train = GT_features_train.append(temp_df, ignore_index=True)
GT_features_train.head()